# Sea Grass  Image Objects Detection
In this project, we develop deep learning based multi object detection model (adaptive YOLOv6). This project detects three objects (leaf, rhizome and root) and its length. 

To complete the project, we used the following steps: 

Step 1: Prepare our dataset in MT-YOLOv6 format

Step 2: Install MT-YOLOv6 dependencies

Step 3: Run MT-YOLOv6 training

Step 4: Evaluate MT-YOLOv6 performance

Step 5: Run MT-YOLOv6 inference on test images

Step 6: OPTIONAL: Deployment

Step 7: OPTIONAL: Active Learning




## Step 1: Easier Dataset Prep

In this model, we used our own prepared dataset from sea-grass images. This dataset was prepared using Roboflow, a set of tools developers use to build better computer vision models quickly and accurately. 100k+ developers use roboflow for (automatic) annotation, converting dataset formats (like to YOLOv6), training, deploying, and improving their datasets/models.

Members of the Roboflow community also share 60M+ images, 90,000+ projects, and 7,000+ pretrained models: https://universe.roboflow.com

### Step 1.1 Access the files from google drive 

In [1]:
# Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [7]:
# Current Project Path. 
# You need to change it based on your path directory.
project_dir = "/content/drive/My Drive/Colab Notebooks/Sea Grass/SeaGrass-main/"

# Make the Directory as working directory 
import os
os.chdir(project_dir)
os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = '/usr/local/lib/python3.9/dist-packages/cv2/qt/plugins/platforms'


### 1.2 Dataset 

We need our dataset in the YOLOv6 format, which requires YOLO TXT annotations, organized directories, and a specific .yaml config file.

If you're following the our dataset example, use the YOLOv6 format data [here](https://app.roboflow.com/cquniversityseagrass/sea_grass/9/export).

You can use any other dataset. If you're preparing your own data, use the guide for creating, formatting, and exporting your custom dataset [here](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/).

For a step-by-step on getting your data into this correct format, follow the blog post [here](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/)


**The YOLOv6 format is as follows:**

[YOLO TXT format](https://roboflow.com/formats/yolo-darknet-txt)
```
# class_id center_x center_y bbox_width bbox_height
1 0.408 0.30266666666666664 0.104 0.15733333333333333
```

Dataset directory format
```
# image directory
path/to/data/images/train/im0.jpg
path/to/data/images/val/im1.jpg
path/to/data/images/test/im2.jpg

# label directory
path/to/data/labels/train/im0.txt
path/to/data/labels/val/im1.txt
path/to/data/labels/test/im2.txt
```

`YAML` format

```
train: ./images/train
val: ./images/valid
test: ./images/test

nc: 3
names: ['leaf','rhizome', 'root']

```

Next, we'll download our dataset in the right format. Use the `meituan/YOLOv6 PyTorch` export. Note that the Meituan implementation requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us. (See [this guide](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/) for more details.)

### Step 1.3 Download prepared data from roboflow 

We'll download our dataset in the right format. Use the meituan/YOLOv6 PyTorch export. Note that the Meituan implementation requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us. (See this [guide](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/) for more details.)




In [ ]:
# REPLACE with your custom code snippet generated above to use your data
# We already download this data from roboflow site thus we do not need to run it. 
# But if you update the dataset then I would like to request you to download the recent data using following code. 

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="j2mtmGReAEJTYKWAJ6Jl")
project = rf.workspace("cquniversityseagrass").project("sea_grass")
dataset = project.version(9).download("mt-yolov6")  # Here 9 is the data version. It may be deffer based on your data version selection. 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 9.4 MB/s 
     |████████████████████████████████| 178 kB 51.9 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 138 kB 41.0 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.17-py3-none-any.whl size=31935 sha256=767a35236b89e846722eaac3cff5396155b7bcbdf9014c31bcbb133410c07f61
  Stored in directory: /root/.cache/pip/wheels/f2/38/3c/b4ac4d8a9d9b44bdcd51f6148ec810b0f05a404e5fed8df48d
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=b47d6c30315641e98e9d752ae515b70cded50efd2d59b523338b550960cfe257
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built roboflow wget
  Attempting un

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Sea_Grass-9 in mt-yolov6:: 100%|██████████| 359/359 [00:02<00:00, 121.69it/s]


# Step 2: Install Dependencies

(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

In [8]:
# Install required libraries
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Step 3: Training Model 
There are a number of ways to fine tune training of YOLOv6, like custom configuration files for fine tuning, multi GPU support, and passing custom training arguments.

### 3.1 Multi GPU Support
YOLOv6 supports single and multi GPU training.

#### 3.1.1 Single GPU:

In [ ]:
#!python tools/train.py --img 640 --batch 32 --epochs 300 --conf configs/yolov6s.py --data Sea_Grass-9/data.yaml --device 0

!python tools/train.py --img 990 --batch 32 --epochs 300 --conf configs/yolov6s.py --data Sea_Grass-9/data.yaml --device 0

Using 1 GPU for training... 
training args are: Namespace(batch_size=32, check_images=False, check_labels=False, conf_file='configs/yolov6s.py', data_path='Sea_Grass-9/data.yaml', device='0', dist_url='env://', epochs=300, eval_final_only=False, eval_interval=20, gpu_count=0, heavy_eval_range=50, img_size=990, local_rank=-1, name='exp', output_dir='./runs/train', rank=-1, resume=False, save_dir='runs/train/exp2', workers=8, world_size=1, write_trainbatch_tb=False)

Train: Final numbers of valid images: 121/ labels: 121. 
1.8s for dataset initialization.
Convert to COCO format
100% 17/17 [00:00<00:00, 58879.58it/s]
Convert to COCO format finished. Resutls saved in Sea_Grass-9/annotations/instances_valid.json
Val: Final numbers of valid images: 17/ labels: 17. 
1.3s for dataset initialization.
Model: Model(
  (backbone): EfficientRep(
    (stem): RepVGGBlock(
      (nonlinearity): ReLU(inplace=True)
      (se): Identity()
      (rbr_dense): Sequential(
        (conv): Conv2d(3, 32, kerne

#### 3.1.2 Multi GPU:

In [ ]:
# We use single GPU, thus multi GPU traning process is inactive here. If you want to use multi GPU use the following training command. 
# !python -m torch.distributed.launch --nproc_per_node 4 tools/train.py --batch 256 --conf configs/yolov6s_finetune.py --data data/data.yaml --device 0,1,2,3

# Step 4: Evaluation
We can evaluate the performance of our custom training using the provided evalution script.

Note we can adjust the below custom arguments.

In [ ]:
# Run evaluation
!python tools/eval.py --batch 32 --data  Sea_Grass-9/data.yaml --img-size 990 --weights runs/train/exp2/weights/best_ckpt.pt --device 0 
# NB: exp1 might be change. Please see the folders in runs/train/ and ensure that laset folder is current exp(i) folder. 

Namespace(batch_size=32, conf_thres=0.001, data='Sea_Grass-9/data.yaml', device='0', half=False, img_size=990, iou_thres=0.65, name='exp', save_dir='runs/val/', task='val', weights='runs/train/exp2/weights/best_ckpt.pt')
Loading checkpoint from runs/train/exp2/weights/best_ckpt.pt

Fusing model...
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to deploy modality.
Model Summary: Params: 17.19M, Gflops: 105.47
Val: Checking formats of labels with 2 process(es): 
17 label(s) found, 0 label(s) missing, 0 label(s) empty, 0 invalid label files: 100% 17/17 [00:00<00:00, 849.13it/s]
Convert to COCO format
100% 17/17 [00:00<00:00, 23195.57it/s]
Convert to COCO format finished. Resutls saved in Sea_Grass-9/annotations/insta

# Step 5: Inference

We can run inference on images of our custom trained model using the provided inference utility.

There are a number of arguments we can adjust:

- --weights, type=str, default='weights/yolov6s.pt', help='model path(s) for inference.')
- --source, type=str, default='data/images', help='the source path, e.g. image-file/dir.')
- --yaml, type=str, default='data/coco.yaml', help='data yaml file.')
- --img-size, type=int, default=640, help='the image-size(h,w) in inference size.')
- --conf-thres, type=float, default=0.25, help='confidence threshold for inference.')
- --iou-thres, type=float, default=0.45, help='NMS IoU threshold for inference.')
- --max-det, type=int, default=1000, help='maximal inferences per image.')
- --device, default='0', help='device to run our model i.e. 0 or 0,1,2,3 or cpu.')
- --save-txt, action='store_true', help='save results to *.txt.')
- --save-img, action='store_false', help='save visuallized inference results.')
- --classes, nargs='+', type=int, help='filter by classes, e.g. --classes 0, or --classes 0 2 3.')
- --agnostic-nms, action='store_true', help='class-agnostic NMS.')
- --project, default='runs/inference', help='save inference results to project/name.')
- -name, default='exp', help='save inference results to project/name.')
- --hide-labels, default=False, action='store_true', help='hide labels.')
- --hide-conf, default=False, action='store_true', help='hide confidences.')
- --half, action='store_true', help='whether to use FP16 half-precision inference.')

We need to pass our custom `.yaml` file so that our label names are correct. We will also pass our `/test` directory to run inference on all images in our test split. In addition, similar to training, we will pass 416x416 images for inference as an example.

In [18]:
# infer on all images in our /test directory
!python tools/infer.py --weights runs/train/exp2/weights/best_ckpt.pt --source Sea_Grass-9/images/test/ --device 0 --name detect_img --save-txt

NumExpr defaulting to 2 threads.
Namespace(weights='runs/train/exp2/weights/best_ckpt.pt', source='Sea_Grass-9/images/test/', yaml='data/coco.yaml', img_size=640, conf_thres=0.25, iou_thres=0.45, max_det=1000, device='0', save_txt=True, save_img=True, save_dir=None, view_img=False, classes=None, agnostic_nms=False, project='runs/inference', name='detect_img', hide_labels=False, hide_conf=False, half=False)
Save directory already existed
Loading checkpoint from runs/train/exp2/weights/best_ckpt.pt

Fusing model...
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to deploy modality.
  0% 0/35 [00:00<?, ?it/s]  img_path  DSCN0988_JPG.rf.0806dcd7df7ea987d0447bb921ce331a.jpg
qt.qpa.xcb: could not connect to display 
qt.q

## Step 5. 1 Test the data

In [ ]:
# display test inference result images
import glob
from IPython.display import Image, display
from google.colab.patches import cv2_imshow, cv2

i = 0
limit = 30 # max images to print
for imageName in glob.glob ('./runs/inference/detect_img/*.jpg'): #assuming JPG
  if i < limit:
    print(imageName)
    cv2.imread(imageName)
    display(Image(filename=imageName))
    # print("\n\n")
  i = i + 1

In [ ]:
!python deploy/ONNX/export_onnx.py --weights runs/train/exp1/weights/best_ckpt.pt --device 0


Namespace(batch_size=1, conf_thres=0.25, device='0', end2end=False, half=False, img_size=[640, 640], inplace=False, iou_thres=0.45, max_wh=None, simplify=False, topk_all=100, trt_version=8, weights='runs/train/exp1/weights/best_ckpt.pt', with_preprocess=False)
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting to export ONNX...
/content/drive/MyDrive/Colab Notebooks/Sea Grass/SeaGrass-main/yolov6/models/effidehead.py:76: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace

#Step 6: OPTIONAL: Deployment

There is a utility included to export the model as ONNX format for deployment as well:

In [ ]:
!python deploy/ONNX/export_onnx.py --weights runs/train/exp1/weights/best_ckpt.pt --device 0


Namespace(batch_size=1, conf_thres=0.25, device='0', end2end=False, half=False, img_size=[640, 640], inplace=False, iou_thres=0.45, max_wh=None, simplify=False, topk_all=100, trt_version=8, weights='runs/train/exp1/weights/best_ckpt.pt', with_preprocess=False)
Loading checkpoint from runs/train/exp1/weights/best_ckpt.pt

Fusing model...
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting to export ONNX...
/content/drive/MyDrive/Colab Notebooks/Sea Grass/SeaGrass-main/yolov6/models/effidehead.py:76: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace

# Step 7: OPTIONAL: Active Learning Example
Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.

In [ ]:
# setup access to your workspace
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="j2mtmGReAEJTYKWAJ6Jl")                               # used above to load data
inference_project =  rf.workspace().project("sea_grass")    # used above to load data
model = inference_project.version(9).model

upload_project = rf.workspace().project("sea_grass")

print("inference reference point: ", inference_project)
print("upload destination: ", upload_project)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 149 kB/s 
     |████████████████████████████████| 178 kB 48.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 145 kB 65.1 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 138 kB 66.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=648c12117111929d5ede9368ab3a98901593f68e0578a4991cc59fee714667ce
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
inference reference point:  {
  "name": "Sea_Grass",
  "type": "object-detection",
  "workspace": "cquniversityseagrass"
}
upload destination:  {
  "name": "Sea_Grass",
  "type": "object-detection",
  "workspace": "cquniversityseagrass"
}
